## Part 1

In [1]:
import pandas as pd

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
df = pd.read_html(wiki)[0]

In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df = df[(df['Borough'] != 'Not assigned')]

In [6]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df.loc[df['Neighbourhood'] == 'Not assigned', 'Borough']

In [7]:
df.shape

(210, 3)

## Part 2

In [10]:
import urllib

In [11]:
urllib.request.urlretrieve ('https://cocl.us/Geospatial_data', 'coordinates.csv')

('coordinates.csv', <http.client.HTTPMessage at 0x192c8c85788>)

In [12]:
coordinates_df = pd.read_csv('coordinates.csv')

In [14]:
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
merged_df = df.merge(coordinates_df, left_on = 'Postcode', right_on = 'Postal Code', how = 'left')

In [29]:
merged_df = merged_df.drop(['Postal Code'], axis=1)

In [30]:
merged_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [ ]:
# Part 3

In [148]:
import requests
import json
import folium
from sklearn.cluster import KMeans
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [145]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [31]:
Toronto_data = merged_df[merged_df['Borough'].str.contains('Toronto')]

In [56]:
print(Toronto_data.shape)
Toronto_data.head()

(74, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
# Foursquare Credentials
CLIENT_ID = '1DMCBGGS2KRXGW5PPJNKX4QZM1Z44LTS4LSFLK4TIG3Y2OMP'
CLIENT_SECRET = 'UTMVOE5VKH2C5YJCZS1BWGCRYG32B0NFQRDWPRTIOO0ZMBGM'
VERSION = '20180605'
LIMIT = 100

In [44]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'], 
                                 latitudes = Toronto_data['Latitude'], 
                                 longitudes = Toronto_data['Longitude'])

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

In [51]:
print(Toronto_venues.shape)
Toronto_venues.head()

(3204, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [52]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Berczy Park,54,54,54,54,54,54
Brockton,23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,36,36,36,36,36,36


In [62]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [64]:
Toronto_onehot = pd.get_dummies (Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [66]:
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

In [67]:
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])

In [69]:
Toronto_onehot = Toronto_onehot[fixed_columns]

In [70]:
Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
Toronto_onehot.shape

(3204, 230)

In [72]:
Toronto_grouped = Toronto_onehot.groupby ('Neighborhood').mean().reset_index()

In [73]:
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Adelaide,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,...,0.0,0.0,0.0,0.00,0.0,0.020000,0.000000,0.0,0.01,0.01
1,Bathurst Quay,0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.00,0.00
2,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.018519,0.000000,0.0,0.00,0.00
3,Brockton,0.043478,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.00,0.00
4,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Underground city,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,...,0.0,0.0,0.0,0.01,0.0,0.010000,0.000000,0.0,0.01,0.00
69,Union Station,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.010000,0.000000,0.0,0.01,0.00
70,University of Toronto,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.027778,0.0,0.00,0.00
71,Victoria Hotel,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,...,0.0,0.0,0.0,0.00,0.0,0.020000,0.000000,0.0,0.01,0.00


In [108]:
num_top_venues = 5

In [109]:
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue', 'freq']
    temp = temp.iloc [1:]
    temp ['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Adelaide----
             Venue  freq
0      Coffee Shop  0.07
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.03
----Bathurst Quay----
              Venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4               Bar  0.06
----Berczy Park----
          Venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Steakhouse  0.04
3          Café  0.04
4      Beer Bar  0.04
----Brockton----
            Venue  freq
0     Coffee Shop  0.09
1       Nightclub  0.09
2            Café  0.09
3  Breakfast Spot  0.09
4     Yoga Studio  0.04
----Business Reply Mail Processing Centre 969 Eastern----
                Venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic Shop  0.06
4         Pizza Place  0.06
----CN Tower----
              Venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  

                Venue  freq
0     Thai Restaurant  0.08
1                Café  0.08
2  Mexican Restaurant  0.08
3                 Bar  0.08
4           Speakeasy  0.04
----Toronto Dominion Centre----
         Venue  freq
0  Coffee Shop  0.12
1         Café  0.08
2        Hotel  0.07
3   Restaurant  0.04
4   Steakhouse  0.04
----Toronto Islands----
                Venue  freq
0         Coffee Shop  0.12
1            Aquarium  0.05
2               Hotel  0.04
3                Café  0.04
4  Italian Restaurant  0.04
----Trinity----
              Venue  freq
0               Bar  0.12
1        Restaurant  0.06
2  Asian Restaurant  0.06
3              Café  0.04
4       Pizza Place  0.04
----Underground city----
         Venue  freq
0  Coffee Shop  0.13
1         Café  0.07
2   Restaurant  0.04
3   Steakhouse  0.04
4          Bar  0.03
----Union Station----
                Venue  freq
0         Coffee Shop  0.12
1            Aquarium  0.05
2               Hotel  0.04
3                Café  0.

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Steakhouse,Bar,Café,Asian Restaurant,Cosmetics Shop,Restaurant,Bakery,Burger Joint,Breakfast Spot
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
2,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cheese Shop,Steakhouse,Café,Greek Restaurant
3,Brockton,Coffee Shop,Breakfast Spot,Café,Nightclub,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Butcher


In [98]:
kclusters = 5

In [100]:
Toronto_grouped_clustering = Toronto_grouped.drop ('Neighborhood', 1)

In [104]:
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Toronto_grouped_clustering)

In [105]:
kmeans.labels_[0:10]

array([1, 4, 1, 1, 1, 4, 1, 1, 1, 1])

In [114]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [118]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Adelaide,Coffee Shop,Steakhouse,Bar,Café,Asian Restaurant,Cosmetics Shop,Restaurant,Bakery,Burger Joint,Breakfast Spot
1,4,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina,Bar
2,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cheese Shop,Steakhouse,Café,Greek Restaurant
3,1,Brockton,Coffee Shop,Breakfast Spot,Café,Nightclub,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
4,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Butcher
...,...,...,...,...,...,...,...,...,...,...,...,...
68,1,Underground city,Coffee Shop,Café,Steakhouse,Restaurant,Burger Joint,Bar,Japanese Restaurant,Deli / Bodega,Hotel,Asian Restaurant
69,1,Union Station,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Sporting Goods Shop,Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery
70,1,University of Toronto,Café,Restaurant,Bookstore,Bar,Bakery,Sandwich Place,Japanese Restaurant,Poutine Place,Dessert Shop,Beer Bar
71,1,Victoria Hotel,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Steakhouse,Bakery,Italian Restaurant,Gastropub,Deli / Bodega


In [119]:
Toronto_merged = Toronto_data

In [123]:
Toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [137]:
Toronto_merged = Toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})

In [138]:
Toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [139]:
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Mexican Restaurant,Hotel,Spa,Ice Cream Shop
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Gym,Yoga Studio,Café,Bar,Fast Food Restaurant,Portuguese Restaurant,Beer Bar,Diner
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Pizza Place,Bakery
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Pizza Place,Bakery
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Breakfast Spot,Park,Cocktail Bar,Cosmetics Shop,Hotel


In [149]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [150]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
158,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant
159,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant
160,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant
161,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant
162,Central Toronto,0,Pub,Coffee Shop,Sushi Restaurant,Supermarket,Sports Bar,Liquor Store,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant


In [151]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Mexican Restaurant,Hotel,Spa,Ice Cream Shop
5,Downtown Toronto,1,Coffee Shop,Park,Gym,Yoga Studio,Café,Bar,Fast Food Restaurant,Portuguese Restaurant,Beer Bar,Diner
12,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Pizza Place,Bakery
13,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Pizza Place,Bakery
26,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Breakfast Spot,Park,Cocktail Bar,Cosmetics Shop,Hotel
35,East Toronto,1,Pizza Place,Pub,Trail,Health Food Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
36,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Cheese Shop,Steakhouse,Café,Greek Restaurant
40,Downtown Toronto,1,Coffee Shop,Ice Cream Shop,Italian Restaurant,Juice Bar,Burger Joint,Café,Japanese Restaurant,Sandwich Place,Bakery,Salad Place
41,Downtown Toronto,1,Grocery Store,Café,Park,Candy Store,Diner,Baby Store,Nightclub,Gas Station,Coffee Shop,Restaurant
48,Downtown Toronto,1,Coffee Shop,Steakhouse,Bar,Café,Asian Restaurant,Cosmetics Shop,Restaurant,Bakery,Burger Joint,Breakfast Spot
